<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/16_Reasoning.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Fill forms automatically
Based on your information, Solar will fill out the form automatically.

In [1]:
! pip3 install -qU langchain langchain-upstage langchain_community python-dotenv

## UPSTAGE_API_KEY
To obtain your Upstage API key, follow these steps:

1. Visit the Upstage AI console at <https://console.upstage.ai>.
2. Sign up for an account if you don't already have one.
3. Log in to your account.
4. Navigate to the API key section.
5. Generate your API key.
6. Copy the key and save it securely.

![Console](./figures/console.upstage.ai.jpg)

In [2]:
# @title set API key
import os
import getpass

from pprint import pprint

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")

In [3]:
my_info = """
John Do, born on April 15, 1985, is a Korean national and currently the CEO of UpstageAI, an AI-driven company based in the United States. He lives in Springfield, IL, at 1234 Maple Street, with his wife Julie Do and their two children, John Do Jr. and Jane Do. Both John and his wife, also born on April 15, 1985, hold Korean passports with expiration dates on September 30, 2025. The family, rooted in their Korean heritage, is well-supported by Korean national health insurance and enjoys strong ties to both Korea and the United States.

Professionally, John earned his Bachelor’s degree in Computer Science from Crestwood University (2004-2008) and an MBA from Brookstone Business School (2010-2012). His career journey saw him start as a Software Developer at InnovateX, where he worked on both front-end and back-end projects. He then moved on to become a Senior Software Engineer at TechCorp, where he led a team of engineers. In 2018, John became the CEO of UpstageAI, where he oversaw its remarkable 150% revenue growth within two years and launched several successful AI products used by Fortune 500 companies.

John is an active member of the international tech community, speaking at conferences and publishing articles in AI journals. He travels frequently, particularly to Europe, where he enjoys annual vacations lasting a week or two. Photography, hiking, reading science fiction novels, and playing the guitar are some of his hobbies. John is also an advocate for emerging technologies, including blockchain and quantum computing, and mentors young entrepreneurs.

In his personal life, John and Julie are dedicated parents to their children, who were born on April 15, 2010, and April 15, 2012. His emergency contact is Julie Do, and they can be reached at sung.kim@example.com or (555) 123-4567. They usually stay at hotels during their European trips, and the purpose of their visits is leisure.

John’s favorite quote is, “The best way to predict the future is to invent it,” by Alan Kay, which he lives by both in his career and personal life.
"""

In [4]:
# France Visit Application Form Fields

form_fields = [
    {
        "Field Name": "Full Name",
        "Description": "Applicant's complete legal name as shown on the passport."
    },
    {
        "Field Name": "Date of Birth",
        "Description": "Applicant's date of birth (DD/MM/YYYY)."
    },
    {
        "Field Name": "Nationality",
        "Description": "Applicant's current nationality."
    },
    {
        "Field Name": "Passport Number",
        "Description": "Unique number of the applicant's passport."
    },
    {
        "Field Name": "Passport Expiry Date",
        "Description": "Expiration date of the passport (DD/MM/YYYY)."
    },
    {
        "Field Name": "Purpose of Visit",
        "Description": "Reason for traveling to France (e.g., tourism, business, study)."
    },
    {
        "Field Name": "Duration of Stay",
        "Description": "Planned length of stay in France (e.g., 2 weeks, 3 months)."
    },
    {
        "Field Name": "Accommodation Details",
        "Description": "Information about where the applicant will be staying in France."
    },
    {
        "Field Name": "Financial Means",
        "Description": "Proof of sufficient funds to cover the stay (e.g., bank statements)."
    },
    {
        "Field Name": "Employment Status",
        "Description": "Current employment status and occupation of the applicant."
    },
    {
        "Field Name": "Travel Insurance",
        "Description": "Details of travel insurance covering medical expenses and emergencies."
    },
    {
        "Field Name": "Previous Visits to France",
        "Description": "Information about any prior visits to France, including dates and purposes."
    },
    {
        "Field Name": "Criminal Record",
        "Description": "Disclosure of any criminal convictions or legal issues."
    },
    {
        "Field Name": "Emergency Contact",
        "Description": "Contact information of a person to be reached in case of emergency."
    },
    {
        "Field Name": "Supporting Documents",
        "Description": "List of additional documents required to support the application (e.g., invitation letter, itinerary)."
    },
    {
        "Field Name": "Travel Itinerary",
        "Description": "Detailed schedule of travel plans while in France."
    },
    {
        "Field Name": "Proof of Accommodation",
        "Description": "Documents confirming hotel reservations or invitation from a host in France."
    },
    {
        "Field Name": "Visa Type",
        "Description": "Specific type of visa being applied for (e.g., short-stay, long-stay)."
    },
    {
        "Field Name": "Health Insurance",
        "Description": "Proof of health insurance coverage during the stay in France."
    },
    {
        "Field Name": "Marital Status",
        "Description": "Applicant's current marital status."
    },
    {
        "Field Name": "Dependents",
        "Description": "Information about any dependents traveling with the applicant."
    },
    {
        "Field Name": "Address in Home Country",
        "Description": "Applicant's residential address in their home country."
    },
    {
        "Field Name": "Contact Information",
        "Description": "Applicant's phone number and email address."
    },
    {
        "Field Name": "Language Proficiency",
        "Description": "Applicant's proficiency in French or other languages."
    }
]

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

form_filling_prompt_teample = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            """You are a form filler. Fill out the form based on the information below.
            If you can guess from personal info, fill it out and make "[GUESS]".
            Do your best to fill out the form.

            If you don't know and no way to guess, just leave it and provide the reason why you left it blank.

            Write the filled form in JSON format.

---  
**Your Information:** 
{my_info}
---   
**Form to Fill:** 
{form_fields}
""",
        )
    ]
)

In [6]:
# write prompt
prompt = form_filling_prompt_teample.format_messages(
    my_info=my_info,
    form_fields=form_fields,
)

print(prompt[0].content)

You are a form filler. Fill out the form based on the information below.
            If you can guess from personal info, fill it out and make "[GUESS]".
            Do your best to fill out the form.

            If you don't know and no way to guess, just leave it and provide the reason why you left it blank.

            Write the filled form in JSON format.

---  
**Your Information:** 

John Do, born on April 15, 1985, is a Korean national and currently the CEO of UpstageAI, an AI-driven company based in the United States. He lives in Springfield, IL, at 1234 Maple Street, with his wife Julie Do and their two children, John Do Jr. and Jane Do. Both John and his wife, also born on April 15, 1985, hold Korean passports with expiration dates on September 30, 2025. The family, rooted in their Korean heritage, is well-supported by Korean national health insurance and enjoys strong ties to both Korea and the United States.

Professionally, John earned his Bachelor’s degree in Computer S

In [7]:
solar_pro = ChatUpstage(model="solar-pro")
solar_mini = ChatUpstage(model="solar-1-mini-chat")

chain_pro = form_filling_prompt_teample | solar_pro | StrOutputParser()
chain_mini = form_filling_prompt_teample | solar_mini | StrOutputParser()


In [8]:

filled_form = chain_pro.invoke(
    {
        "my_info": my_info,
        "form_fields": form_fields,
    }
)

print(filled_form)


{
  "Full Name": "John Do",
  "Date of Birth": "15/04/1985",
  "Nationality": "Korean",
  "Passport Number": "[GUESS]",
  "Passport Expiry Date": "30/09/2025",
  "Purpose of Visit": "Leisure",
  "Duration of Stay": "2 weeks",
  "Accommodation Details": "[GUESS]",
  "Financial Means": "[GUESS]",
  "Employment Status": "CEO of UpstageAI",
  "Travel Insurance": "[GUESS]",
  "Previous Visits to France": "[GUESS]",
  "Criminal Record": "None",
  "Emergency Contact": {
    "Name": "Julie Do",
    "Email": "sung.kim@example.com",
    "Phone": "(555) 123-4567"
  },
  "Supporting Documents": "[GUESS]",
  "Travel Itinerary": "[GUESS]",
  "Proof of Accommodation": "[GUESS]",
  "Visa Type": "Short-stay",
  "Health Insurance": "Korean national health insurance",
  "Marital Status": "Married",
  "Dependents": "John Do Jr., Jane Do",
  "Address in Home Country": "1234 Maple Street, Springfield, IL",
  "Contact Information": {
    "Phone Number": "[GUESS]",
    "Email": "john.do@example.com"
  },
  "L

In [9]:
university_application_form = [
    {
        "Field Name": "Full Name",
        "Description": "Applicant's complete legal name as shown on official documents."
    },
    {
        "Field Name": "Date of Birth",
        "Description": "Applicant's date of birth (DD/MM/YYYY)."
    },
    {
        "Field Name": "Gender",
        "Description": "Applicant's gender identity."
    },
    {
        "Field Name": "Nationality",
        "Description": "Applicant's current nationality."
    },
    {
        "Field Name": "Email Address",
        "Description": "Applicant's primary email address for correspondence."
    },
    {
        "Field Name": "Phone Number",
        "Description": "Applicant's contact phone number."
    },
    {
        "Field Name": "Mailing Address",
        "Description": "Applicant's current mailing address."
    },
    {
        "Field Name": "Intended Major",
        "Description": "The field of study the applicant wishes to pursue."
    },
    {
        "Field Name": "Degree Level",
        "Description": "The level of degree the applicant is applying for (e.g., Bachelor's, Master's, PhD)."
    },
    {
        "Field Name": "High School Name",
        "Description": "Name of the high school the applicant graduated from."
    },
    {
        "Field Name": "High School Graduation Date",
        "Description": "Date of high school graduation (MM/YYYY)."
    },
    {
        "Field Name": "GPA",
        "Description": "Applicant's Grade Point Average from their most recent academic institution."
    },
    {
        "Field Name": "Standardized Test Scores",
        "Description": "Scores from relevant standardized tests (e.g., SAT, ACT, GRE, TOEFL, IELTS)."
    },
    {
        "Field Name": "Extracurricular Activities",
        "Description": "List of the applicant's extracurricular activities, clubs, and organizations."
    },
    {
        "Field Name": "Work Experience",
        "Description": "Relevant work experience, internships, or volunteer positions."
    },
    {
        "Field Name": "Awards and Honors",
        "Description": "Any academic or extracurricular awards or honors received."
    },
    {
        "Field Name": "Personal Statement",
        "Description": "A brief essay about the applicant's background, goals, and reasons for applying."
    },
    {
        "Field Name": "References",
        "Description": "Names and contact information for academic or professional references."
    },
    {
        "Field Name": "Financial Aid",
        "Description": "Indication of whether the applicant is applying for financial aid or scholarships."
    },
    {
        "Field Name": "Language Proficiency",
        "Description": "Applicant's proficiency in the language of instruction and any other relevant languages."
    },
    {
        "Field Name": "Previous University Education",
        "Description": "Details of any previous university-level education, if applicable."
    },
    {
        "Field Name": "Emergency Contact",
        "Description": "Name and contact information for an emergency contact person."
    },
]

In [10]:
filled_form = chain_mini.invoke(
    {
        "my_info": my_info,
        "form_fields": university_application_form,
    }
)

print(filled_form)

{
  "Full Name": "John Do",
  "Date of Birth": "15/04/1985",
  "Gender": "[GUESS]",
  "Nationality": "Korean",
  "Email Address": "[GUESS]",
  "Phone Number": "[GUESS]",
  "Mailing Address": "1234 Maple Street, Springfield, IL",
  "Intended Major": "[GUESS]",
  "Degree Level": "[GUESS]",
  "High School Name": "[GUESS]",
  "High School Graduation Date": "[GUESS]",
  "GPA": "[GUESS]",
  "Standardized Test Scores": "[GUESS]",
  "Extracurricular Activities": "[GUESS]",
  "Work Experience": "[GUESS]",
  "Awards and Honors": "[GUESS]",
  "Personal Statement": "John Do is a Korean national and currently the CEO of UpstageAI, an AI-driven company based in the United States. He earned his Bachelor’s degree in Computer Science from Crestwood University and an MBA from Brookstone Business School. He has a career history in software development and engineering, and is an active member of the international tech community, speaking at conferences and publishing articles in AI journals. He is an advo

In [11]:
filled_form = chain_pro.invoke(
    {
        "my_info": my_info,
        "form_fields": university_application_form,
    }
)

print(filled_form)

{
  "Full Name": "John Do",
  "Date of Birth": "15/04/1985",
  "Gender": "[GUESS]",
  "Nationality": "Korean",
  "Email Address": "sung.kim@example.com",
  "Phone Number": "(555) 123-4567",
  "Mailing Address": "1234 Maple Street, Springfield, IL",
  "Intended Major": "[GUESS]",
  "Degree Level": "[GUESS]",
  "High School Name": "[GUESS]",
  "High School Graduation Date": "[GUESS]",
  "GPA": "[GUESS]",
  "Standardized Test Scores": "[GUESS]",
  "Extracurricular Activities": "Photography, hiking, reading science fiction novels, playing the guitar, mentoring young entrepreneurs",
  "Work Experience": "Software Developer at InnovateX, Senior Software Engineer at TechCorp, CEO of UpstageAI",
  "Awards and Honors": "Led UpstageAI to 150% revenue growth within two years, launched several successful AI products used by Fortune 500 companies",
  "Personal Statement": "[GUESS]",
  "References": "Julie Do, email: sung.kim@example.com, phone: (555) 123-4567",
  "Financial Aid": "[GUESS]",
  "Lang

In [12]:
company_leave_form = [
    {
        "Field Name": "Employee Name",
        "Description": "Full name of the employee requesting leave."
    },
    {
        "Field Name": "Employee ID",
        "Description": "Unique identification number assigned to the employee."
    },
    {
        "Field Name": "Department",
        "Description": "Department or division where the employee works."
    },
    {
        "Field Name": "Position",
        "Description": "Employee's current job title or position."
    },
    {
        "Field Name": "Leave Type",
        "Description": "Type of leave being requested (e.g., Annual, Sick, Maternity, Paternity, Bereavement, etc.)."
    },
    {
        "Field Name": "Reason for Leave",
        "Description": "Brief explanation for the leave request."
    },
    {
        "Field Name": "Contact During Leave",
        "Description": "Phone number or email where the employee can be reached during leave, if necessary."
    },
    {
        "Field Name": "Emergency Contact",
        "Description": "Name and contact information for an emergency contact person."
    },
   
]

In [13]:
filled_form = chain_mini.invoke(
    {
        "my_info": my_info,
        "form_fields": company_leave_form,
    }
)

print(filled_form)

{
  "Employee Name": "John Do",
  "Employee ID": "[GUESS]",
  "Department": "[GUESS]",
  "Position": "CEO",
  "Leave Type": "[GUESS]",
  "Reason for Leave": "[GUESS]",
  "Contact During Leave": "[GUESS]",
  "Emergency Contact": "Julie Do, sung.kim@example.com, (555) 123-4567"
}


In [15]:
filled_form = chain_pro.invoke(
    {
        "my_info": my_info,
        "form_fields": company_leave_form,
    }
)

print(filled_form)

{
  "Employee Name": "John Do",
  "Employee ID": "[GUESS]",
  "Department": "UpstageAI",
  "Position": "CEO",
  "Leave Type": "[GUESS]",
  "Reason for Leave": "[GUESS]",
  "Contact During Leave": "sung.kim@example.com or (555) 123-4567",
  "Emergency Contact": "Julie Do, sung.kim@example.com or (555) 123-4567"
}
